# Text Generation
## Character-Level LSTM in PyTorch

In this assignment, you will train a Reccurent Neural Network to generate a text one character at the time. The task is divided into steps for simpler navigation.

Let's start!

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

##Step 0. Data

As training data, we will use "War and Peace" by Leo Tolstoy. The book in plain text format can be downloaded from [Project Gutenberg website ](https://www.gutenberg.org/ebooks/2600).

Download and load data.

In [2]:
# open text file and read in data as `text`
with open('2600-0.txt', 'r') as f:
    text = f.read()

Let's view the first 100 symbols:

In [3]:
text[:100]

'\ufeffThe Project Gutenberg eBook of War and Peace, by Leo Tolstoy\n\nThis eBook is for the use of anyone a'

The book starts at `7277`:

In [4]:
text[7277:7500]

'TER II\n\n    CHAPTER III\n\n    CHAPTER IV\n\n    CHAPTER V\n\n    CHAPTER VI\n\n    CHAPTER VII\n\n    CHAPTER VIII\n\n    CHAPTER IX\n\n    CHAPTER X\n\n    CHAPTER XI\n\n    CHAPTER XII\n\n\n\n\n\n\n\n\n\n\nBOOK ONE: 1805\n\n\n\n\n\nCHAPTER I\n\n“Well, Princ'

##Step 1. Tokenization (15 points)

Now let's create dictionaries for converting characters to integers and vice versa. This makes it easier to use symbols as input on the neural network.

In [5]:
# Create two dictionaries:
# int2char -- maps integers to characters
# char2int -- maps characters to unique integers

## YOUR CODE HERE
unique_chars=list(set(text))
char2int={c:i for i,c in enumerate(unique_chars)}
int2char={i:c for c,i in char2int.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

Let's see how the encoding of characters into integers looks like

In [6]:
encoded[:100]

array([ 29,   7,   4,  12,  68,  77,   0,  81,  60,  12,  98,  13,  68,
        22,  48,  13,  12,  49,  80,  12,   0,  35,  68,  12, 107,  81,
        81,  86,  68,  81,  39,  68,  20,  59,   0,  68,  59,  49,  78,
        68,  77,  12,  59,  98,  12, 106,  68,  80,  16,  68,  82,  12,
        81,  68,   7,  81,  88,  72,  13,  81,  16,  84,  84,   7,   4,
        99,  72,  68,  12, 107,  81,  81,  86,  68,  99,  72,  68,  39,
        81,   0,  68,  13,   4,  12,  68,  48,  72,  12,  68,  81,  39,
        68,  59,  49,  16,  81,  49,  12,  68,  59])

##Step 2. One-hot encoding (15 points)

LSTM expects one-hot encoded input, which means that each character is converted to an integer (via our created dictionary) and then converted to a vector, where a value 1 will be only on a corresponding position and the rest of the vector will be filled with zeros.

Implement a function that does this kind of coding.


In [7]:
def one_hot_encode(arr, n_labels):
    
    # arr - array of integers
    # n_labels - number of labels (the size of a one-hot-encoded vector)

    ## YOUR CODE HERE
    
    r,c=arr.shape
    one_hot=np.zeros((r,c,n_labels))
    for row in range(len(arr)):
        for col in range(len(arr[row])):
            one_hot[row][col][arr[row][col]]=1
    
    return one_hot

In [8]:
# check that the function works correctly
test_indx = np.array([[7, 2, 5]])
one_hot = one_hot_encode(test_indx, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]


The output must be

```
[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]]]
```

##Step 3. Mini-batches (20 points)

To train the neural network, we will organize mini-batches as follows: divide the entire input sequence 'arr' by the desired number of subsequences (parameter `batch_size`), and send a sequence of length `seq_length` to the input of the network.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>


### How to create mini-batches


1.   Find the total number of batches `n_batches` that fit the text.
2.   Discard all unnecessary text that does not fit into full batches.
3.   Split text into `n_batches` batches'.
4.   Get `x's` and `y`'s for a batch. Therefore, `y` is a version of `x` shifted by 1.



In [9]:
def get_batches(arr, batch_size, seq_length):
    # Create a generator that returns batches of size batch_size x seq_length
    
    # print(arr.shape)
    batch_size_total = batch_size * seq_length
    ## Get the number of batches we can make
    n_batches = len(arr)//batch_size_total## YOUR CODE HERE
    
    ## Keep only enough characters to make full batches
    arr = arr[:batch_size_total*n_batches]## YOUR CODE HERE
    
    ## Reshape into batch_size rows
    arr = arr.reshape(batch_size, -1) ## YOUR CODE HERE
    # arr=np.hstack((arr, arr[:,0].reshape(batch_size, 1)))
    
    ## Iterate over the batches using a window of size seq_length
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:,n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

Let's check how our function works.

In [10]:
batches = get_batches(encoded, 4, 30)
x, y = next(batches)

In [11]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[29  7  4 12 68 77  0 81 60 12]
 [ 0 26 72 84  4 12 59 88 13  4]
 [80 12  0 99 49 35 68  4 99 72]
 [59 49 78 88 12 68 74 99 13  4]]

y
 [[ 7  4 12 68 77  0 81 60 12 98]
 [26 72 84  4 12 59 88 13  4 68]
 [12  0 99 49 35 68  4 99 72 68]
 [49 78 88 12 68 74 99 13  4 68]]


If you've implemented get_batches correctly, the result should look something like this (exact numbers may vary):
```
x
 [[ 48  94  12 110  32  96 107  34  18 106]
 [ 96  78   1  69  32  34  18  34  27   3]
 [  3  96   0  18  96  86  18 100  34  96]
 [ 32  96 110  42 101  96   0  18  19  96]]

y
 [[ 94  12 110  32  96 107  34  18 106  32]
 [ 78   1  69  32  34  18  34  27   3  94]
 [ 96   0  18  96  86  18 100  34  96   1]
 [ 96 110  42 101  96   0  18  19  96   3]]
 ```
Make sure the data is shifted one step for `y`.

##Step 4. Defining the network (40 points)

### Model Architecture

The proposed architecture is as follows:

* Define an [LSTM layer](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html#torch.nn.LSTM) with `dropout=drop_prob` and `batch_first=True` (since we use batches)
* Define a [Dropout layer](https://pytorch.org/docs/stable/generated/torch.nn.Dropout.html#torch.nn.Dropout) with `drop_prob`.
* Define a [Linear layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear) with `in_features=n_hidden` and `out_features` equals to number of characters.

In [12]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU


In [13]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define the LSTM layer
        ## YOUR CODE HERE
        self.lstm=nn.LSTM(input_size=len(self.chars),hidden_size=n_hidden, num_layers=n_layers, dropout=drop_prob, batch_first=True)

        # Define a dropout layer
        ## YOUR CODE HERE
        self.dropout=nn.Dropout(p=drop_prob)

        # Define the final, fully-connected output layer
        ## YOUR CODE HERE
        self.linear=nn.Linear(in_features=n_hidden, out_features=len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        # Get the outputs and the new hidden state from the lstm
        ## YOUR CODE HERE
        raw_out, hidden=self.lstm(x.float(), hidden)

        # Pass through a dropout layer
        ## YOUR CODE HERE
        out=self.dropout(raw_out)

        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        ## YOUR CODE HERE
        out=out.contiguous().view(-1, self.n_hidden)

        # Put x through the fully-connected layer
        ## YOUR CODE HERE
        out=self.linear(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

##Step 5. Train the model (10 points)

We use Cross Entropy as a loss function, Adam as optimizer, and [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) to get rid of the gradient explosion.

In [14]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

Initialize the model and set hyperparameters.

In [15]:
# Set your model hyperparameters

n_hidden = 384## YOUR CODE HERE
n_layers = 4## YOUR CODE HERE

net = CharRNN(unique_chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(112, 384, num_layers=4, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (linear): Linear(in_features=384, out_features=112, bias=True)
)


Set training hyperparameters.

In [16]:
batch_size = 256## YOUR CODE HERE
seq_length = 150## YOUR CODE HERE
# start small if you are just testing initial behavior
n_epochs = 100## YOUR CODE HERE

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

Epoch: 1/100... Step: 10... Loss: 3.2485... Val Loss: 3.1346
Epoch: 1/100... Step: 20... Loss: 3.2192... Val Loss: 3.0966
Epoch: 1/100... Step: 30... Loss: 3.1947... Val Loss: 3.0912
Epoch: 1/100... Step: 40... Loss: 3.1760... Val Loss: 3.0911
Epoch: 1/100... Step: 50... Loss: 3.1598... Val Loss: 3.0874
Epoch: 1/100... Step: 60... Loss: 3.1556... Val Loss: 3.0862
Epoch: 1/100... Step: 70... Loss: 3.1436... Val Loss: 3.0862
Epoch: 2/100... Step: 80... Loss: 3.1424... Val Loss: 3.0850
Epoch: 2/100... Step: 90... Loss: 3.1447... Val Loss: 3.0855
Epoch: 2/100... Step: 100... Loss: 3.1481... Val Loss: 3.0859
Epoch: 2/100... Step: 110... Loss: 3.1547... Val Loss: 3.0860
Epoch: 2/100... Step: 120... Loss: 3.1452... Val Loss: 3.0851
Epoch: 2/100... Step: 130... Loss: 3.1388... Val Loss: 3.0850
Epoch: 2/100... Step: 140... Loss: 3.1356... Val Loss: 3.0833
Epoch: 2/100... Step: 150... Loss: 3.1332... Val Loss: 3.0848
Epoch: 3/100... Step: 160... Loss: 3.1198... Val Loss: 3.0829
Epoch: 3/100... S

##Step 6. Making Predictions

Now that the model is trained, we want to predict the next character for the sequence of input symbols. We pass a character as input, and the network predicts the next character. Then we take that character, pass it as input, and get another predicted character, and so on.

### Details of a `predict`  function

Our RNN's output comes from a fully connected layer and outputs the **distribution of the next character scores**. To actually get the next character, we use the softmax function, which gives us a **probability** distribution that we can then choose to predict the next character.

**Top K sampling**

Our predictions are based on a categorical distribution of probabilities for all possible characters. We can make the sampling process smarter by looking at only some of the most likely $K$ characters. This will prevent the network from giving us completely absurd characters, and will also allow some noise and randomness to be introduced into the selected text. This technique is called [top K](https://pytorch.org/docs/stable/generated/torch.topk.html#torch.topk) sampling.

In [17]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

To generate the text, you need to feed the initial characters, let's call them `prime`.

In [18]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [20]:
print(sample(net, 1000, prime='Anna', top_k=5))

Annal was the despention of the white too world, and then told him
that they all were being and
wine to tell a smile. She was to see an incomerent times and told how the moming he went to the sound with him on his
cap,
but whenere the crowd of thought and soldiers were something, with his sabers.

A sharply at once showed the carriage at that
still, began speaking
whether he was as is always astonies who
to see the same thousand things are down. At his colonel and answered all the distance.

His feeling
along the cloth, and the figure of the sense of the commander in
chief and a count and the strength through his horse. To see it would be and had answered
a mind with steps, as if they were a cold officer’s attractory of what he always all the forests, who should have any
staff officer with
stern tones and a country, straight at the state of the clear where would say, he was the conditions of the carpace, holding off. The prince said this with the position was all a made the feelings of